In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas
import torch
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
import os

In [3]:
from torchsummary import summary

In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
# BATCH_SIZE = 16
PATH_SAVE = "/content/gdrive/MyDrive/models/"
PATH = "/content/gdrive/MyDrive/training/"
PATH_TESTING = "/content/gdrive/MyDrive/testing/"

In [5]:
# Run after copying
PATH = "/content/training/training"


In [ ]:
!unzip "/content/gdrive/MyDrive/training.zip" -d "/content"

In [6]:
class GeM(nn.Module):
    def __init__(self, p=3.0, eps=1e-6):
        super(GeM, self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return F.adaptive_avg_pool2d(x.clamp(min=self.eps).pow(self.p), (1, 1)).pow(1. / self.p)

In [7]:
landmark_dict = None
def load_landmark_dict():
    global landmark_dict
    if landmark_dict is None:
        landmark_dict = eval(open("/content/gdrive/MyDrive/nu stergeti/landmarks.json").read())


def get_classes() -> int:
    load_landmark_dict()
    return int([*landmark_dict.keys()][-1]) + 1

In [8]:
def load_model():
    j = 0
    model = get_res_net_ibn(classes=get_classes())
    for i in range(1,1000):
        if os.path.isfile(f"{PATH_SAVE}model.ibn.{i}.model"):
            print(f"Reached checkpoint {i}")
            model = torch.load(f"{PATH_SAVE}model.ibn.{i}.model")
            j = i
        else:
            break
    return model, j


def get_res_net_ibn(classes: int = 491):
    model = torch.hub.load('XingangPan/IBN-Net',
                           'resnet50_ibn_a', 
                           pretrained=True
                          #  , map_location=torch.device('cpu')
                          , force_reload = True
                           )
    previous_layer = model.fc.in_features
    model.avgpool = GeM()
    model.fc = torch.nn.Linear(previous_layer, classes)
    return model

In [9]:
def train(epoch, dataloader, model, device, lr=0.0005):
    size = len(dataloader.dataset)
    print(f"Dataset size = {size}")

    train_loss = 0
    correct = 0

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = CrossEntropyLoss()
    
    for X, y in tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()

        outputs = model(X)
        loss = criterion(outputs, y)

        train_loss += loss.item()
        pred = outputs.argmax(dim=1, keepdim=True)
        correct += pred.eq(y.view_as(pred)).sum().item()

        loss.backward()
        optimizer.step()
        model.zero_grad()

    train_loss /= len(dataloader.dataset)
    print(
        f'Epoch:{epoch}\t Average loss: {train_loss:.4f}, Accuracy: {100. * correct / len(dataloader.dataset):.0f}%\n')



def test(epoch, model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = CrossEntropyLoss()
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test Epoch: {} \tAverage loss: {:.4f}, Accuracy: {:.0f}%\n'.format(epoch, test_loss, 100. * correct / len(
        test_loader.dataset)))


In [10]:
model, epoch = load_model()
# print(model)
# summary(model.cuda(), (3, 224, 224))
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
print(f"Already trained {epoch} epochs")
model = model.to(device)

Downloading: "https://github.com/XingangPan/IBN-Net/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/XingangPan/IBN-Net/releases/download/v1.0/resnet50_ibn_a-d9d0bb7b.pth" to /root/.cache/torch/hub/checkpoints/resnet50_ibn_a-d9d0bb7b.pth


  0%|          | 0.00/97.7M [00:00<?, ?B/s]

Reached checkpoint 1
Reached checkpoint 2
Reached checkpoint 3
Reached checkpoint 4
Reached checkpoint 5
Reached checkpoint 6
Reached checkpoint 7
Reached checkpoint 8
Using cuda device
Already trained 8 epochs


In [11]:
def get_data_loader():
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )])

    dataset = datasets.ImageFolder(PATH, transform=transform)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return dataloader

In [ ]:
dataloader = get_data_loader()


In [ ]:
# TO COMMENT IF epoch > 10
for x in list(model.children()):
    x.training = False

model.fc.training = True
model.avgpool.training = True
# END COMMENT

In [ ]:
copy_epoch = epoch
for i in range(copy_epoch + 1, copy_epoch + 10):
    epoch += 1
    print(f"Epoch: {epoch}")
    train(dataloader=dataloader, model=model, device=device, epoch=i)
    torch.save(model, f"{PATH_SAVE}model.ibn.{epoch}.model")
    # test(epoch=i, test_loader=testloader, model=model, device=device)

Epoch: 6
Dataset size = 135045


100%|██████████| 2111/2111 [1:15:08<00:00,  2.14s/it]


Epoch:6	 Average loss: 0.0028, Accuracy: 95%

Epoch: 7
Dataset size = 135045


100%|██████████| 2111/2111 [1:14:22<00:00,  2.11s/it]


Epoch:7	 Average loss: 0.0023, Accuracy: 96%

Epoch: 8
Dataset size = 135045


100%|██████████| 2111/2111 [1:14:07<00:00,  2.11s/it]


Epoch:8	 Average loss: 0.0019, Accuracy: 96%

Epoch: 9
Dataset size = 135045


  1%|▏         | 31/2111 [01:04<1:11:45,  2.07s/it]

In [ ]:
epoch

In [12]:
!unzip "/content/gdrive/MyDrive/validation.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/validation/40088/c3db1c1ce83a36dc.jpg  
  inflating: /content/validation/40088/c440c71e3deb96f1.jpg  
  inflating: /content/validation/40088/c7662230a4ce9203.jpg  
  inflating: /content/validation/40088/cd08ea84e97ae97f.jpg  
  inflating: /content/validation/40088/cecf425d31e60907.jpg  
  inflating: /content/validation/40088/cf08d890e53bc2e6.jpg  
  inflating: /content/validation/40088/cfede9eeef643fb9.jpg  
  inflating: /content/validation/40088/d0c419a5884f269c.jpg  
  inflating: /content/validation/40088/d10ff949f46b7ba7.jpg  
  inflating: /content/validation/40088/d26101355612e754.jpg  
  inflating: /content/validation/40088/d6406caca1bd7c80.jpg  
  inflating: /content/validation/40088/dad3bb095f023ea7.jpg  
  inflating: /content/validation/40088/dc8f87c79fceb024.jpg  
  inflating: /content/validation/40088/de223999c2df7a55.jpg  
  inflating: /content/validation/40088/df28a3c011027002.jpg  
  inflating: /conte

In [16]:
PATH_TESTING = "/content/validation"

In [17]:
def get_test_loader():
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )])
    dataset = datasets.ImageFolder(PATH_TESTING, transform=transform)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    return dataloader

In [18]:
testloader = get_test_loader()

In [19]:
test(epoch=epoch, test_loader=testloader, model=model, device=device)

100%|██████████| 263/263 [05:01<00:00,  1.15s/it]

Test Epoch: 8 	Average loss: 0.0029, Accuracy: 95%



In [21]:
print(model.cpu())

ResNet_IBN(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck_IBN(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): IBN(
        (IN): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (BN): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [22]:
torch.save(model.cpu(), f"{PATH_SAVE}model.cpu.95.model")